In [5]:
# EE488C Special Topics in EE <Deep Learning and AlphaGo>
# Fall 2016, School of EE, KAIST

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)
sess = tf.InteractiveSession()
n_first_filter = 32 # original = 30
n_second_filter = 64 # original = 50
n_hidden_node = 1024 # original = 500
n_output_node = 10


# Input layer
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,28,28,1])

# First convolutional layer
W_conv = tf.Variable(tf.truncated_normal([5, 5, 1, n_first_filter], stddev=0.1))
b_conv = tf.Variable(tf.constant(0.1, shape=[n_first_filter]))
h_conv = tf.nn.conv2d(x_image, W_conv, strides=[1, 1, 1, 1], padding='VALID') 
h_relu = tf.nn.relu(h_conv + b_conv)
h_pool = tf.nn.max_pool(h_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Second convolutional layer
W_conv2 = tf.Variable(tf.truncated_normal([3, 3, n_first_filter, n_second_filter], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[n_second_filter]))
h_conv2 = tf.nn.conv2d(h_pool, W_conv2, strides=[1, 1, 1, 1], padding='VALID')
h_relu2 = tf.nn.relu(h_conv2 + b_conv2)
h_pool2 = tf.nn.max_pool(h_relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Fully-connected Layer
W_fc1 = tf.Variable(tf.truncated_normal([5 * 5 * n_second_filter, n_hidden_node], stddev=0.1)) # so, 5 * 5 * n_second_filter is a final representation of data
b_fc1 = tf.Variable(tf.constant(0.1, shape=[n_hidden_node]))
h_pool_flat = tf.reshape(h_pool2, [-1, 5*5*n_second_filter])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = tf.Variable(tf.truncated_normal([n_hidden_node, n_output_node], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[n_output_node]))
y_hat=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# Train and Evaluate the Model
cross_entropy = - tf.reduce_sum(y_*tf.log(y_hat))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_hat,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fb410412150>> ignored


In [7]:
sess.run(tf.initialize_all_variables())
print("=================================")
print("|Epoch\tBatch\t|Train\t|Val\t|")
print("|===============================|")
for j in range(20):
    for i in range(550):
        batch = mnist.train.next_batch(100)
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.7})
        if i%50 == 49:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.})
            val_accuracy = accuracy.eval(feed_dict=\
                {x: mnist.validation.images, y_:mnist.validation.labels, keep_prob: 1.})
            print("|%d\t|%d\t|%.4f\t|%.4f\t|"%(j+1, i+1, train_accuracy, val_accuracy))
print("=================================")
test_accuracy = accuracy.eval(feed_dict=\
    {x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.})
print("test accuracy=%.4f"%(test_accuracy))

|Epoch	Batch	|Train	|Val	|
|===============================|
|1	|50	|0.8300	|0.7912	|
|1	|100	|0.8800	|0.8854	|
|1	|150	|0.8600	|0.9132	|
|1	|200	|0.9200	|0.9288	|
|1	|250	|0.8700	|0.9348	|
|1	|300	|0.9200	|0.9400	|
|1	|350	|0.9200	|0.9472	|
|1	|400	|0.9400	|0.9540	|
|1	|450	|0.9600	|0.9552	|
|1	|500	|0.9400	|0.9592	|
|1	|550	|0.9800	|0.9592	|
|2	|50	|0.9500	|0.9588	|
|2	|100	|0.9700	|0.9652	|
|2	|150	|0.9500	|0.9666	|
|2	|200	|0.9700	|0.9682	|
|2	|250	|0.9700	|0.9660	|
|2	|300	|0.9500	|0.9702	|
|2	|350	|0.9900	|0.9720	|
|2	|400	|0.9600	|0.9720	|
|2	|450	|0.9800	|0.9726	|
|2	|500	|0.9800	|0.9714	|
|2	|550	|0.9900	|0.9718	|
|3	|50	|0.9600	|0.9758	|
|3	|100	|0.9900	|0.9760	|
|3	|150	|0.9400	|0.9760	|
|3	|200	|0.9900	|0.9780	|
|3	|250	|1.0000	|0.9772	|
|3	|300	|0.9800	|0.9778	|
|3	|350	|0.9700	|0.9772	|
|3	|400	|0.9800	|0.9800	|
|3	|450	|0.9800	|0.9804	|
|3	|500	|0.9600	|0.9806	|
|3	|550	|0.9900	|0.9796	|
|4	|50	|0.9800	|0.9808	|
|4	|100	|0.9600	|0.9808	|
|4	|150	|0.9800	|0.9822	|
|4	|200